工場 (x, y) から商品を店 (a, b, c) に配送します。

供給量、需要量、輸送コストが下表で与えられているとき、総輸送コストが最小となる配送の仕方を求めてください。


|        | 店a | 店b | 店c | 供給量 |
|     -  |   - |   - |   - |     - |
| 工場x  |  10 |   6 |  16 |     8 |
| 工場y  |   8 |   8 |   4 |    16 |
| 需要量 |  12 |   4 |   8 |       |

(http://www.nct9.ne.jp/m_hiroi/light/pulp01.html#chap03)

In [1]:
import pulp
import pandas as pd

In [2]:
df = pd.DataFrame(
    columns=["店a", "店b", "店c", "供給量"],
    index=["工場x", "工場y", "需要量"],
    data=[
        [10, 6, 16,  8],
        [ 8, 8,  4, 16],
        [12, 4,  8]
    ]
)
df

,店a,店b,店c,供給量
工場x,10,6,16,8.0
工場y,8,8,4,16.0
需要量,12,4,8,NaN


In [3]:
problem = pulp.LpProblem(name="transportation", sense=pulp.LpMinimize)
problem

transportation:
MINIMIZE
None
VARIABLES

In [4]:
factories = df.index[:-1].values.tolist()
shops = df.columns[:-1].values.tolist()
table = pulp.LpVariable.dicts("", (factories, shops), lowBound=0, cat=pulp.LpInteger)
table

{'工場x': {'店a': _工場x_店a, '店b': _工場x_店b, '店c': _工場x_店c},
 '工場y': {'店a': _工場y_店a, '店b': _工場y_店b, '店c': _工場y_店c}}

In [5]:
problem += pulp.lpSum([[table[f][s] for s in shops] for f in factories])
problem

transportation:
MINIMIZE
1*_工場x_店a + 1*_工場x_店b + 1*_工場x_店c + 1*_工場y_店a + 1*_工場y_店b + 1*_工場y_店c + 0
VARIABLES
0 <= _工場x_店a Integer
0 <= _工場x_店b Integer
0 <= _工場x_店c Integer
0 <= _工場y_店a Integer
0 <= _工場y_店b Integer
0 <= _工場y_店c Integer

In [6]:
for f in factories:
    problem += pulp.lpSum([table[f][s] for s in shops]) <= df["供給量"][f]
problem

transportation:
MINIMIZE
1*_工場x_店a + 1*_工場x_店b + 1*_工場x_店c + 1*_工場y_店a + 1*_工場y_店b + 1*_工場y_店c + 0
SUBJECT TO
_C1: _工場x_店a + _工場x_店b + _工場x_店c <= 8

_C2: _工場y_店a + _工場y_店b + _工場y_店c <= 16

VARIABLES
0 <= _工場x_店a Integer
0 <= _工場x_店b Integer
0 <= _工場x_店c Integer
0 <= _工場y_店a Integer
0 <= _工場y_店b Integer
0 <= _工場y_店c Integer

In [7]:
for s in shops:
    problem += pulp.lpSum([table[f][s] for f in factories]) >= df[s]["需要量"]
problem

transportation:
MINIMIZE
1*_工場x_店a + 1*_工場x_店b + 1*_工場x_店c + 1*_工場y_店a + 1*_工場y_店b + 1*_工場y_店c + 0
SUBJECT TO
_C1: _工場x_店a + _工場x_店b + _工場x_店c <= 8

_C2: _工場y_店a + _工場y_店b + _工場y_店c <= 16

_C3: _工場x_店a + _工場y_店a >= 12

_C4: _工場x_店b + _工場y_店b >= 4

_C5: _工場x_店c + _工場y_店c >= 8

VARIABLES
0 <= _工場x_店a Integer
0 <= _工場x_店b Integer
0 <= _工場x_店c Integer
0 <= _工場y_店a Integer
0 <= _工場y_店b Integer
0 <= _工場y_店c Integer

In [8]:
status = problem.solve(pulp.PULP_CBC_CMD(msg=0))
print("Status", pulp.LpStatus[status])

Status Optimal


In [9]:
print("=" * 3, "Assignment", "=" * 25)
print('総輸送コスト: {}'.format(pulp.value(problem.objective)))
print("-" * 40)
print("配送計画")
for f in factories:
    for s in shops:
        print(f, "to", s, ":", int(table[f][s].value()))
print("=" * 40)

=== Assignment =========================
総輸送コスト: 24.0
----------------------------------------
配送計画
工場x to 店a : 0
工場x to 店b : 4
工場x to 店c : 4
工場y to 店a : 12
工場y to 店b : 0
工場y to 店c : 4
